In [4]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 11.1/11.1 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for en_core_web_sm: filename=en_core_web_sm-2.1.0-py3-none-any.whl size=11074421 sha256=dcf218a40fcda1785905df3aff0a7717bb7381041707c54f49d5a26486788041
  Stored in directory: C:\Users\gbala\AppData\Local\Temp\pip-ephem-wheel-cache-yrg37ew1\wheels\59\4f\8c\0dbaab09a776d1fa3740e9465078bfd903cc22f3985382b496
Successfully built en_core_web_sm
[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [5]:
import spacy
spacy.__version__

'2.1.8'

In [6]:
from nlg.utils import load_spacy_model
from nlg import grammar
from nlg.search import templatize
import pandas as pd
from gramex.data import filter as gfilter

In [481]:
df_2=pd.read_csv("dummy.csv") # Dummy data
df_2['Achieved'] = df_2['Success Percentage'].apply(lambda x: "achieved" if x > 99.6 else "missed")
df_2['Month'] = pd.to_datetime(df_2['Month_Year'], format='%b-%y')

In [482]:
df_2

,Country,Channel,Month_Year,Successful,UnSuccessful,Total,Success Percentage,Achieved,Month
0,HK,Internet,Dec-23,673480,16516,689996,90.90,missed,2023-12-01
1,HK,Internet,Jan-24,630152,17966,648118,95.80,missed,2024-01-01
2,HK,Internet,Feb-24,777223,18763,795986,99.87,achieved,2024-02-01
3,HK,Mobile,Dec-23,6751532,238105,6989637,99.90,achieved,2023-12-01
4,HK,Mobile,Jan-24,6063108,224842,6287950,99.87,achieved,2024-01-01
5,HK,Mobile,Feb-24,7002254,235435,7237689,99.99,achieved,2024-02-01


In [484]:
fh_args = {
  '_by': ['Channel','Month','Month_Year','Achieved'],
    '_sort':['Channel','-Month'] #Sorting based on Channel and Month (Descending Order)
    
}

In [485]:
xdf=gfilter(df_2,fh_args.copy())

In [486]:
xdf

,Channel,Month,Month_Year,Achieved,Successful|sum,UnSuccessful|sum,Total|sum,Success Percentage|sum
2,Internet,2024-02-01,Feb-24,achieved,777223,18763,795986,99.87
1,Internet,2024-01-01,Jan-24,missed,630152,17966,648118,95.80
0,Internet,2023-12-01,Dec-23,missed,673480,16516,689996,90.90
5,Mobile,2024-02-01,Feb-24,achieved,7002254,235435,7237689,99.99
4,Mobile,2024-01-01,Jan-24,achieved,6063108,224842,6287950,99.87
3,Mobile,2023-12-01,Dec-23,achieved,6751532,238105,6989637,99.90


In [487]:
nlp=load_spacy_model()

In [488]:
text=nlp("Feb-24 Month target is Achieved for Mobile Channel")
nugget=templatize(text,fh_args,df_2)

C:\Users\gbala\anaconda3\envs\temp_ai\lib\site-packages\nlg\search.py:65: UserWarning: Ignoring lemmatization.
  warnings.warn('Ignoring lemmatization.')
C:\Users\gbala\anaconda3\envs\temp_ai\lib\site-packages\nlg\search.py:87: UserWarning: Cannot lemmatize multi-word cells.
  warnings.warn('Cannot lemmatize multi-word cells.')


In [489]:
nugget

{% set fh_args = {"_by": ["Channel", "Month", "Month_Year", "Achieved"], "_sort": ["Channel", "-Month"]}  %}
{% set df = U.gfilter(orgdf, fh_args.copy()) %}
{% set fh_args = U.sanitize_fh_args(fh_args, orgdf) %}
{# Do not edit above this line. #}
{{ df["Month_Year"].iloc[3] }} {{ df.columns[1] }} target is {{ fh_args['_by'][3] }} for {{ df["Channel"].iloc[-1] }} {{ df.columns[0] }}

In [493]:
print(nugget.render(df_2))

b'    Feb-24 Month target is Achieved for Mobile Channel'


In [494]:
nugget.variables

{Month: {{ df.columns[1] }},
 Channel: {{ df.columns[0] }},
 Mobile: {{ df["Channel"].iloc[-1] }},
 Achieved: {{ fh_args['_by'][3] }},
 Feb-24: {{ df["Month_Year"].iloc[3] }}}

In [495]:
#Modifying Variables for correct format
var=nugget.get_var("Feb-24")
var.set_expr('df["Month_Year"].iloc[0]')
var


{{ df["Month_Year"].iloc[0] }}

In [496]:
#Modified achieved column for correct format
var_token = text[-4]
nugget.add_var(var_token, expr='df[df["Channel"]=="Mobile"]["Achieved"].iloc[0]')

In [497]:
nugget.variables

{Month: {{ df.columns[1] }},
 Channel: {{ df.columns[0] }},
 Mobile: {{ df["Channel"].iloc[-1] }},
 Achieved: {{ df[df["Channel"]=="Mobile"]["Achieved"].iloc[0] }},
 Feb-24: {{ df["Month_Year"].iloc[0] }}}

In [498]:
print(nugget.render(df_2))

b'    Feb-24 Month target is achieved for Mobile Channel'


In [499]:
text=nlp("Feb-24 Month target is Achieved for Mobile Channel")

In [ ]:
######################### Testing ###################

In [509]:
df_2=pd.read_csv("dummy.csv")

In [510]:
#Let decrease success percentage for Mobile Channel and month feb-24 and test 
df_2

,Country,Channel,Month_Year,Successful,UnSuccessful,Total,Success Percentage
0,HK,Internet,Dec-23,673480,16516,689996,90.90
1,HK,Internet,Jan-24,630152,17966,648118,95.80
2,HK,Internet,Feb-24,777223,18763,795986,99.87
3,HK,Mobile,Dec-23,6751532,238105,6989637,99.90
4,HK,Mobile,Jan-24,6063108,224842,6287950,99.87
5,HK,Mobile,Feb-24,7002254,235435,7237689,99.99


In [511]:
df_2['Success Percentage'].iloc[5]=90
df_2

C:\Users\gbala\anaconda3\envs\temp_ai\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,Country,Channel,Month_Year,Successful,UnSuccessful,Total,Success Percentage
0,HK,Internet,Dec-23,673480,16516,689996,90.90
1,HK,Internet,Jan-24,630152,17966,648118,95.80
2,HK,Internet,Feb-24,777223,18763,795986,99.87
3,HK,Mobile,Dec-23,6751532,238105,6989637,99.90
4,HK,Mobile,Jan-24,6063108,224842,6287950,99.87
5,HK,Mobile,Feb-24,7002254,235435,7237689,90.00


In [512]:
df_2['Achieved'] = df_2['Success Percentage'].apply(lambda x: "achieved" if x > 99.6 else "missed")
df_2['Month'] = pd.to_datetime(df_2['Month_Year'], format='%b-%y')

In [513]:
print(nugget.render(df_2))

b'    Feb-24 Month target is missed for Mobile Channel'
